In [1]:
from datasets import load_dataset
import json

/home/neel/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import re
import random
import os

In [3]:
# strg + shift + 7 for multi lines comment

In [4]:
# Get the current working directory (where the notebook is running)
script_dir = os.getcwd()

# Construct the path to your folder (assuming it's inside the repo)
dataset_folder_path = os.path.join(script_dir, "datasets")

# # Read the file
# with open(file_path, "r") as f:
#     content = f.read()
#     print(content)


In [5]:
# Access a file inside that folder
input_file = os.path.join(dataset_folder_path, "winogrande_validation.jsonl")
output_file = os.path.join(dataset_folder_path, "winogrande_val_splitup.jsonl")

In [6]:
import json
import re

# Load the dataset (assuming JSONL format)
def load_winogrande(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

# Save the processed dataset
def save_dataset(data, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item) + "\n")

# Function to split a sentence, prioritizing "." (not at end), "than", then other punctuation
def split_sentence(sentence):
    # Prioritize "." but only if it's not at the end
    match = re.search(r"\.(?!$)", sentence)  # Ensures the period is not at the end
    if match:
        idx = match.start() + 1
        return sentence[:idx], sentence[idx:].strip()

    # If no ".", prioritize other punctuation (; , :)
    match = re.search(r"([,;:])", sentence)
    if match:
        idx = match.start() + 1  # Keep punctuation in part1
        return sentence[:idx], sentence[idx:].strip()

    # If "than" exists, split after the compared entity
    idx = sentence.find(" than ")
    if idx != -1:
        next_space = sentence.find(" ", idx + 6)  # Look after " than "
        if next_space != -1:
            return sentence[:next_space], sentence[next_space:].strip()

    # If no punctuation or "than", split in half at a word boundary
    mid = len(sentence) // 2
    while mid > 0 and sentence[mid] != " ":
        mid -= 1
    if mid == 0:
        mid = len(sentence) // 2
        while mid < len(sentence) and sentence[mid] != " ":
            mid += 1
    
    return sentence[:mid].strip(), sentence[mid:].strip()

# Process dataset
def process_dataset(dataset):
    new_data = []
    for item in dataset:
        sentence = item["sentence"]
        part1, part2 = split_sentence(sentence)
        new_data.append({"part1": part1, "part2": part2, **item})  # Keep original fields
    return new_data


In [9]:
import pandas as pd

In [11]:
input_file= os.path.join(dataset_folder_path, "winogrande_val_splitup.jsonl")

In [12]:
dataset = load_winogrande(input_file)
pd.DataFrame(dataset)

,part1,part2,sentence,option1,option2,answer
0,Sarah was a much better surgeon than Maria,so _ always got the easier cases.,Sarah was a much better surgeon than Maria so ...,Sarah,Maria,2
1,Sarah was a much better surgeon than Maria,so _ always got the harder cases.,Sarah was a much better surgeon than Maria so ...,Sarah,Maria,1
2,They were worried the wine would ruin the bed ...,but the _ was't ruined.,They were worried the wine would ruin the bed ...,blanket,bed,2
3,Terry tried to bake the eggplant in,the toaster oven but the _ was too big.,Terry tried to bake the eggplant in the toaste...,eggplant,toaster,1
4,"At night,",Jeffrey always stays up later than Hunter to w...,"At night, Jeffrey always stays up later than H...",Jeffrey,Hunter,1
...,...,...,...,...,...,...
1262,Kenneth was really scared to have surgery but ...,_ was under a lot of stress.,Kenneth was really scared to have surgery but ...,Kenneth,Neil,1
1263,Joel went to Michael 's house and consumed a l...,so _ had to ride back home as a passenger.,Joel went to Michael 's house and consumed a l...,Joel,Michael,1
1264,"At the new restaurant, Betty convinced Lindsey...",_ was relieved when she tried new things.,"At the new restaurant, Betty convinced Lindsey...",Betty,Lindsey,1
1265,Jane received a pet tortoise and an aquarium a...,but the _ was too small.,Jane received a pet tortoise and an aquarium a...,aquarium,tortoise,1


In [35]:
dataset = load_winogrande(input_file)
processed_data = process_dataset(dataset)  # Randomly choose method
save_dataset(processed_data, output_file)

print(f"Processed dataset saved to {output_file}")

Processed dataset saved to /home/neel/Desktop/Data-Comtamination-Sem-3-project/datasets/winogrande_val_splitup.jsonl


In [4]:
## for CB dataset
import pandas as pd

In [5]:
cb_file = os.path.join(dataset_folder_path, "CommitmentBank-items.csv")

In [ ]:
# Load the CSV file (update 'your_file.csv' with the actual file path)
df = pd.read_csv(cb_file)

# Extract relevant columns
df_context = df[['uID', 'Context']]
df_embedding = df[['uID', 'Embedding']]
df_target = df[['uID', 'Target']]

# Save to separate CSV files
df_context.to_csv("context_sentences.csv", index=False)
df_embedding.to_csv("prompt_sentences.csv", index=False)
df_target.to_csv("target_sentences.csv", index=False)

In [6]:
df = pd.read_csv(cb_file)[['uID', 'Embedding','Context', 'Target']].to_csv("cb_sentences.csv", index=False)
